In [1]:
from experiments.experiments_results import rank_results, filter_exp
s, r = rank_results(comp_col='arch', based_on='nb_lags', rank_by='score')

C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\M

In [2]:
s

,rank1,rank2,rank3,rank4
rnn,0,0,2,2
cnn,2,2,0,0
tcn,2,2,0,0
mlp,0,0,2,2


In [3]:
r

,bo,best_rank_by,rank1,rank2,rank3,rank4
0,1,"[0.0132, 0.0168, 0.0376, 0.0376]",cnn,tcn,rnn,mlp
1,2,"[0.0122, 0.016, 0.0298, 0.0306]",tcn,cnn,rnn,mlp
2,4,"[0.0128, 0.0156, 0.0294, 0.0302]",cnn,tcn,mlp,rnn
3,8,"[0.012, 0.0132, 0.028, 0.0354]",tcn,cnn,mlp,rnn


In [4]:
filter_exp(nb_lags=8)

,poi,arch,user,gran,nb_lags,period,nb_params,mean_score,mean_time,score_0,score_1,score_2,score_3,score_4,time_0,time_1,time_2,time_3,time_4
7665,imp,tcn,39,60,8,1,49921,0.0120,2.5684,0.009,0.014,0.011,0.013,0.013,1.365,2.096,2.557,2.993,3.831
6513,imp,cnn,39,60,8,1,168321,0.0132,0.1226,0.009,0.011,0.014,0.017,0.015,0.059,0.094,0.093,0.122,0.245
7666,imp,tcn,39,60,8,2,49921,0.0150,2.7164,0.015,0.018,0.012,0.019,0.011,1.322,2.114,2.935,3.538,3.673
6525,imp,cnn,39,30,8,1,168321,0.0170,0.2266,0.022,0.008,0.030,0.013,0.012,0.096,0.126,0.300,0.201,0.410
6514,imp,cnn,39,60,8,2,168321,0.0170,0.1146,0.018,0.013,0.018,0.019,0.017,0.059,0.076,0.106,0.128,0.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9034,imp,mlp,50,60,8,2,9281,NaN,0.4036,0.053,0.049,0.038,NaN,0.073,0.199,0.367,0.565,0.000,0.887
9047,imp,mlp,50,30,8,4,9281,NaN,1.3806,0.084,0.081,0.060,NaN,0.154,0.600,1.237,1.942,0.000,3.124
9106,imp,mlp,54,60,8,2,9281,NaN,0.4170,0.166,0.125,0.223,NaN,0.107,0.207,0.403,0.601,0.000,0.874
9107,imp,mlp,54,60,8,4,9281,NaN,0.3964,0.334,0.117,0.230,NaN,0.105,0.195,0.383,0.558,0.000,0.846


In [3]:
from data_analysis.inference import *
df = generate_MET_stadistics()

In [5]:
df['corr'].mean()

0.8689655682297882